# Automating the boring things

In the previous chapters we initialized our image calculator library, did some planning (both short and long term), and started implementing some new features with some unit tests to make sure it is working as expected. We finished the previous chapter by submitting a Pull Request that ended up merging into our development branch `dev`.

Even if we didn't ask for any review (as we are working alone in this tutorial), it could be interesting to swap our position with the one of the reviewer. Imagine you just got tagged to review our previous PR. When you receive the notification, chances are that you have absolutely no clue what the hell is going on in the code. Even the big picture might be missing in your brain.

Because our PR is incremental and linked to a detailed issue, you, as a reviewer, can start by refreshing your mind by reading the issue (and potentially related issues...). Once the big picture of why we are doing that and how we agreed on doing it, then you can come back to the PR.

Before diving into the code diff, you might wonder whether the code is:

- well formatted (meaning that it follows conventions from the team, which can be inspired by usual rules like PEP8)
- not breaking anything
- well tested and functional (meaning that new tests are passing)

## Automatically formatting the code and checking the syntax

The formatting part is one of the most boring thing to do as a reviewer ! Imagine yourself tracking all lines exceding some length, spotting forgotten white spaces between operators, or chasing trailing whitespaces... Your time is clearly more important than that ! Moreover, your review could take ages and discourage the contributor with thousands of meaningless stuffs to change.

Luckily, there are tools that can take care of that for us ! The idea is simple: we configure the tool to give it our conventions (for example with don't want lines longer than 88 characters), we use it to automatically format our local files before committing them, and we commit our tool configuration to version control such that other contributors can use it as well. If all contributors are using the same tool with the same configuration, then the code base will be totally consistent and discussions about formatting won't even exist in PRs.

These tools are called code formatters and there exists multiple solutions for Python. A direct extension to formatters are called *code linters* and are focused on analyzing the code for potential errors, bugs, stylistic errors, or weird constructs. They detect a whole range of bugs like syntax errors and are extremely useful when used with formatters.

In the Aramis team, we are mostly using [Ruff](https://docs.astral.sh/ruff/), which is a super fast code formatter and linter for Python. It is written in Rust and is orders of magnitude faster than older tools such as Black for example.

Let's install Ruff as always, with Poetry:

```bash
$ poetry add ruff --group dev
Using version ^0.12.0 for ruff

Updating dependencies
Resolving dependencies... (0.8s)

Package operations: 1 install, 0 updates, 0 removals

  - Installing ruff (0.12.0): Downloading... 87%
  - Installing ruff (0.12.0)

Writing lock file
```

You can check that Ruff was added to the dev dependencies in the `pyproject.toml` file:

```toml
[tool.poetry.group.dev.dependencies]
pytest = "^8.4.1"
pytest-cov = "^6.2.1"
ruff = "^0.12.0"
```

As mentioned above, we need to give our conventions to Ruff such that it can perform its work. Of course, Ruff comes with good defaults such that we don't have much to configure. You can add the following at the end of your `pyproject.toml` file:

```toml
[tool.ruff]
line-length = 88

[tool.ruff.lint]
select = ["E", "F", "W", "I"]
ignore = ["E501"]

[tool.ruff.lint.isort]
known-first-party = ["calculator"]

[tool.ruff.format]
quote-style = "double"
indent-style = "space"
```

Feel free to take a closer look at the documentation of Ruff to better understand these rules and see the extent of what is possible with Ruff.

Now that we have Ruff installed and configured, we can check our code base:

```bash
$ poetry run ruff check .
tests/unittests/image_test.py:1:1: I001 [*] Import block is un-sorted or un-formatted
  |
1 | / import numpy as np
2 | | from calculator.image import Image
  | |__________________________________^ I001
  |
  = help: Organize imports

Found 1 error.
[*] 1 fixable with the `--fix` option.
```

Ruff found on error in one of our tests. It looks like it is unhappy with the imports inside one of our test function but it also tells us it is able to automatically fix the problem for us, we only need to pass the `--fix` option:

```bash
$ poetry run ruff check --fix .
Found 1 error (1 fixed, 0 remaining).
```

We can look at the diff of our test file to see that Ruff automatically added a line in between the numpy import and the internal import of `Image`. It is indeed a recommendation to structure your imports in such a way. We can already see how Ruff is going to save a lot of our time by fixing boring things like this.

```bash
$ git diff tests/unittests/image_test.py
diff --git a/tests/unittests/image_test.py b/tests/unittests/image_test.py
index a38faaa..2357ed4 100644
--- a/tests/unittests/image_test.py
+++ b/tests/unittests/image_test.py
@@ -1,4 +1,5 @@
 import numpy as np
+
 from calculator.image import Image
```

```bash
$ git add poetry.lock pyproject.toml tests/unittests/image_test.py
$ git commit -m "Add and configure Ruff, sanitize code base"
$ git push origin dev
```